<a href="https://colab.research.google.com/github/EmilSeyfullayev/Credit_Risk_Modeling_in_Python/blob/main/Scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
import scipy.stats as stat
import pickle

In [2]:
class LogisticRegression_with_p_values:
    
    def __init__(self,*args,**kwargs):#,**kwargs):
        self.model = linear_model.LogisticRegression(*args,**kwargs)#,**args)

    def fit(self,X,y):
        self.model.fit(X,y)
        
        #### Get p-values for the fitted model ####
        denom = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T
        F_ij = np.dot((X / denom).T,X) ## Fisher Information Matrix
        Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.model.coef_[0] / sigma_estimates # z-score for eaach model coefficient
        p_values = [stat.norm.sf(abs(x)) * 2 for x in z_scores] ### two tailed test for p-values
        
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        #self.z_scores = z_scores
        self.p_values = p_values
        #self.sigma_estimates = sigma_estimates
        #self.F_ij = F_ij

In [3]:
loaded_model = pickle.load(
    open("/content/drive/MyDrive/Credit Risk Modeling/finalized_model.sav", 'rb')
)

In [5]:
lg = loaded_model.model

In [6]:
lg.coef_

array([[ 1.19920637e+00,  9.60957332e-01,  7.67240462e-01,
         5.70963385e-01,  3.80889638e-01,  1.67464526e-01,
        -7.95648717e-02,  1.22875514e-02,  1.00498574e-01,
        -9.45489143e-03, -6.57554625e-03,  6.34974819e-02,
         4.11168564e-02,  7.89735506e-02,  7.10854980e-02,
         1.09126370e-01,  1.65443616e-01,  2.51127875e-01,
         2.86188138e-01,  3.47177716e-01,  4.69843690e-01,
         2.82473158e-02,  6.76828867e-02,  1.01134699e-01,
         2.56517267e-01,  1.09592631e-01,  1.20311033e-01,
         9.38237093e-02,  7.78912376e-02,  1.24816816e-01,
         8.84818724e-01,  5.48146777e-01,  3.55308290e-01,
         2.74362336e-01,  9.38522181e-02, -6.58080441e-02,
        -5.95261401e-02, -2.65132095e-02,  4.89959467e-02,
         1.73325993e-02,  6.96502436e-01,  5.49928171e-01,
         3.10185462e-01,  2.58323574e-01,  3.78817296e-01,
         4.55189983e-01,  7.03743872e-02,  5.46436509e-02,
         1.84450640e-02,  4.15778285e-02,  4.94760746e-0